User Query
   ↓
Agent 1 (PDF RAG)

   ├─ Answer found → RETURN

   └─ Not found → Trigger Agent 2 
   
                     ↓
               Agent 2 (Wikipedia)


In [1]:
from crewai import LLM
from dotenv import load_dotenv
import os
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")


llm = LLM(
    model="llama-3.1-8b-instant",
    api_key= GROQ_API_KEY,temperature=0.7
)

# llm = LLM(
#     model = "huggingface/meta-llama/Llama-3.3-70B-Instruct",
#     # model="huggingface/meta-llama/Llama-3.1-8B-Instruct",
    
#     api_key= os.getenv("HUGGINGFACEHUB_API_TOKEN")
# ) 

In [4]:
from crewai import Agent, Crew, Task, LLM
from crewai_tools import PDFSearchTool


# PDFSearchTool ব্যবহার করুন - এটি দ্রুত
pdf_tool = PDFSearchTool(
    pdf='ragagent/data/MachineLearning.pdf',
    config={
        "embedding_model": {
            "provider": "sentence-transformer",
            "config": {"model": "all-MiniLM-L6-v2"}
        },
        "vectordb": {
            "provider": "chromadb",
            "config": {}
        }
    }
)


/home/ahmed/miniconda3/envs/aiapp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from crewai.utilities.paths import db_storage_path
import os

storage_path = db_storage_path()
print(f"CrewAI storage location: {storage_path}")

# সব ফোল্ডার দেখুন
if os.path.exists(storage_path):
    for item in os.listdir(storage_path):
        print(f"📁 {item}")

CrewAI storage location: /home/ahmed/snap/code/219/.local/share/Agent
📁 chroma.sqlite3
📁 chromadb-b6e1f73bd8c9480e42e90856fc17defb.lock
📁 .crewai_user.json
📁 ee9e4d22-5caf-4327-a98e-5574ad9bf99c
📁 latest_kickoff_task_outputs.db


In [ ]:
import chromadb
from crewai.utilities.paths import db_storage_path
import os
import shutil

# CrewAI storage path খুঁজুন
storage_path = db_storage_path()
print(f"Storage path: {storage_path}")

# # সম্পূর্ণ storage ডিলিট করুন
# if os.path.exists(storage_path):
#     shutil.rmtree(storage_path)
#     print("✅ Storage deleted!")

In [11]:
from crewai.utilities.paths import db_storage_path
import os

storage_path = db_storage_path()
print(f"CrewAI storage location: {storage_path}")

# সব ফোল্ডার দেখুন
if os.path.exists(storage_path):
    for item in os.listdir(storage_path):
        print(f"📁 {item}")

CrewAI storage location: /home/ahmed/snap/code/219/.local/share/Agent
📁 chroma.sqlite3
📁 chromadb-b6e1f73bd8c9480e42e90856fc17defb.lock
📁 .crewai_user.json
📁 ee9e4d22-5caf-4327-a98e-5574ad9bf99c
📁 latest_kickoff_task_outputs.db


In [6]:
agent = Agent(
    role="PDF Knowledge Expert",
    goal="Answer questions based on PDF documents.",
    backstory="An AI assistant that extracts information from PDFs.",
    llm=llm,
    tools=[pdf_tool],  # tools হিসেবে দিন, knowledge_sources নয়
    # verbose=True
)


# -------------------------
# Task
# -------------------------
pdf_task = Task(
    description=(
        "Answer the user query using ONLY the PDF content.\n"
        "Rules:\n"
        "1. Do NOT use prior knowledge\n"
        "2. Do NOT guess\n"
        "3. If the PDF does not contain the answer, respond with: NOT_FOUND\n\n"
        "User Question: {input}"
    ),
    agent=agent,
    expected_output="Answer from PDF or NOT_FOUND"
)


# -------------------------
# Crew
# -------------------------
pdf_crew = Crew(
    agents=[agent],
    tasks=[pdf_task],
    # verbose=True
)


In [22]:
result = pdf_crew.kickoff(inputs={"input": "Decision Tree Learning?"})
result.raw

'3.3 Decision Tree Learning\nA decision tree is an acyclic graph that can be used to make decisions. In each branching\nnode of the graph, a specific feature j of the feature vector is examined. If the value of the\nfeature is below a specific threshold, then the left branch is followed; otherwise, the right\nbranch is followed. As the leaf node is reached, the decision is made about the class to which\nthe example belongs.\nAs the title of the section suggests, a decision tree can be learned from data.\n3.3.1 Problem Statement\nLike previously, we have a collection of labeled examples; labels belong to the set {0, 1}. We\nwant to build a decision tree that would allow us to predict the class of an example given a\nfeature vector.\n3.3.2 Solution\nThere are various formulations of the decision tree learning algorithm. In this book, we\nconsider just one, called ID3.\nThe optimization criterion, in this case, is the average log-likelihood:\n1\nN\nN\nÿ\ni=1\nyi ln fID3(xi) + (1 ≠yi) ln (

In [23]:
result = pdf_crew.kickoff(inputs={"input": "Semi-Supervised Learning?"})
result.raw

'1.2.3 Semi-Supervised Learning\n\nIn semi-supervised learning, the dataset contains both labeled and unlabeled examples.\n\nUsually, the quantity of unlabeled examples is much higher than the number of labeled\n\nexamples. The goal of a semi-supervised learning algorithm is the same as the goal of\n\nthe supervised learning algorithm. The hope here is that using many unlabeled examples can\n\nhelp the learning algorithm to ﬁnd (we might say “produce” or “compute”) a better model2.\n\n2It could look counter-intuitive that learning could beneﬁt from adding more unlabeled examples. It seems\n\nlike we add more uncertainty to the problem. However, when you add unlabeled examples, you add more\n\ninformation about your problem: a larger sample reﬂects better the probability distribution the data we\n\nlabeled came from. Theoretically, a learning algorithm should be able to leverage this additional information.'

In [24]:
result = pdf_crew.kickoff(inputs={"input": "whats is agentic ai?"})
result.raw

'NOT_FOUND'

In [7]:
result = pdf_crew.kickoff(inputs={"input": "whats is Linear Regression?"})
result.raw

'3\n\nFundamental Algorithms\n\nIn this chapter, I describe ﬁve algorithms which are not just the most known but also either\n\nvery eﬀective on their own or are used as building blocks for the most eﬀective learning\n\nalgorithms out there.\n\n3.1\n\nLinear Regression\n\nLinear regression is a popular regression learning algorithm that learns a model which is a\n\nlinear combination of features of the input example.\n\n3.1.1\n\nProblem Statement\n\nWe have a collection of labeled examples {(xi, yi)}N\n\ni=1, where N is the size of the collection,\n\nxi is the D-dimensional feature vector of example i = 1, . . . , N, yi is a real-valued1 target\n\nand every feature x(j)\n\ni , j = 1, . . . , D, is also a real number.\n\nWe want to build a model fw,b(x) as a linear combination of features of example x:\n\nfw,b(x) = wx + b,\n\n(1)\n\nwhere w is a D-dimensional vector of parameters and b is a real number. The notation fw,b\n\nmeans that the model f is parametrized by two values: w and b.\

## agent2

In [8]:
from crewai import LLM

# # Example: Flan-T5 large (free hosted)
# llm = LLM(
#     model="google/flan-t5-large",
#     api_key=HUGGINGFACEHUB_API_TOKEN,
#     temperature=0,   # deterministic output
#     max_tokens=1000  # optional
# )
llm = LLM(
    model = "huggingface/meta-llama/Llama-3.3-70B-Instruct",
    # model="huggingface/meta-llama/Llama-3.1-8B-Instruct",
    api_key= os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    temperature=0,   # deterministic output
    max_tokens=1000  # optional
)

In [3]:

from crewai.tools import tool
from crewai_tools import SerperDevTool
import wikipedia

serper_search = SerperDevTool()

@tool("Wikipedia Search")
def wikipedia_search(query: str) -> str:
    """
    Searches Wikipedia and returns a short summary (up to 4 sentences).
    """
    try:
        summary = wikipedia.summary(query, sentences=4)
        return summary
    except wikipedia.DisambiguationError as e:
        # If multiple options exist, pick first
        return wikipedia.summary(e.options[0], sentences=4)
    except Exception:
        return "NOT_FOUND"

In [5]:
from crewai import Agent, Crew, Task, LLM

# -------------------------
# Wikipedia Agent
# -------------------------
wikipedia_agent = Agent(
    llm=llm,
    role="Wikipedia Research Agent",
    goal="Answer user questions using Wikipedia if PDF agent did not find an answer",
    backstory=(
        "You are an expert in summarizing Wikipedia content. "
        "Always give short, factual answers. "
        "If the topic is not found, respond with exactly: NOT_FOUND"
    ),
    tools=[wikipedia_search],
    # verbose=True,
)

# -------------------------
# Task
# -------------------------
wikipedia_task = Task(
    description=(
        "Answer the user query using Wikipedia content.\n"
        "Rules:\n"
        "1. Search Wikipedia using the wikipedia_search tool.\n"
        "2. Provide concise and factual summary.\n"
        "3. If the topic is not found, respond with: NOT_FOUND\n"
        "User Question: {input}"
    ),
    agent=wikipedia_agent,
    expected_output="Answer from Wikipedia or NOT_FOUND"
)

# -------------------------
# Crew
# -------------------------
wikipedia_crew = Crew(
    agents=[wikipedia_agent],
    tasks=[wikipedia_task],
    # verbose=True
)



In [18]:
result = wikipedia_crew.kickoff(inputs={"input": "whats is Linear Regression?"})
result.raw

'In statistics, linear regression is a model that estimates the relationship between a scalar response (dependent variable) and one or more explanatory variables (regressor or independent variable). A model with exactly one explanatory variable is a simple linear regression; a model with two or more explanatory variables is a multiple linear regression. This term is distinct from multivariate linear regression, which predicts multiple correlated dependent variables rather than a single dependent variable.\nIn linear regression, the relationships are modeled using linear predictor functions whose unknown model parameters are estimated from the data. Most commonly, the conditional mean of the response given the values of the explanatory variables (or predictors) is assumed to be an affine function of those values; less commonly, the conditional median or some other quantile is used. Like all forms of regression analysis, linear regression focuses on the relationship between two or more v

In [ ]:
result = wikipedia_crew.kickoff(inputs={"input": "whats is agentic ai?"})
result.raw

## combine 2 agent def 

In [20]:
def run_combined_crew(query: str):
    # Run PDF Agent first
    pdf_result = pdf_crew.kickoff(inputs={"input": query})
    pdf_answer = pdf_result.raw.strip()

    if pdf_answer == "NOT_FOUND":
        # Fallback to Wikipedia Agent
        wiki_result = wikipedia_crew.kickoff(inputs={"input": query})
        return {
            "source": "Wikipedia",
            "answer": wiki_result.raw
        }
    else:
        return {
            "source": "PDF",
            "answer": pdf_answer
        }

result = run_combined_crew("whats is agentic ai?")

In [21]:
result

{'source': 'Wikipedia',
 'answer': 'There is no Wikipedia page on "Agentic AI" with a detailed content, however, In the context of generative artificial intelligence, AI agents (also referred to as compound AI systems or agentic AI) are a class of intelligent agents distinguished by their ability to operate autonomously in complex environments. Agentic AI tools prioritize decision-making over content creation and do not require human prompts or continuous oversight.\n\n== Overview ==\nAI agents possess several key attributes, including complex goal structures, natural language interfaces, the capacity to act independently of user supervision, and the integration of software tools or planning systems. Their control flow is frequently driven by large language models (LLMs). Agents also include memory systems for remembering previous user-agent interactions and orchestration software for organizing agent components.'}